## Laboratorium 8.1


## Metody wykrywania ruchu: przepływ optyczny (_optical flow_)

### Wstęp

Na poprzednich laboratoriach poznaliśmy podstawowe metody przetwarzania obrazów, a więc dwuwymiarowych sygnałów przestrzennych. Jednak w praktyce czasami dysponujemy materiałem wideo, a więc sygnałami _trójwymiarowymi_, gdzie trzecim wymiarem jest czas. Oczywiście, można takie dane traktować jako po prostu sekwencję niezależnych obrazów - i wtedy działają wszystkie poznane dotychczas metody. Jednak dlaczego by nie wykorzystać tej dziedziny czasowej do przetwarzania sygnału? Intuicyjnie czujemy, że jeśli jakiś rejon obrazu przesunął się w czasie, to prawdopodobnie ma on inne _znaczenie_ niż rejon, który pozostał w tym samym miejscu, lub przesunął w innym kierunku czy z inną szybkością. Jeśli zatem jesteśmy w stanie wykryć to przesunięcie - czyli po prostu _ruch_ - to otrzymamy pewną informację o zawartości obrazu (wideo). Na przykład, będziemy w stanie oddzielić poruszające się obiekty od stacjonarnego tła, albo oddzielić inaczej poruszające się obiekty.

Najpowszechniejszą metodą do automatycznej detekcji ruchu jest metoda przepływu optycznego (ang. _optical flow_). Pominiemy w tej instrukcji matematyczne podstawy tej metody - te są wystarczająco dobrze wyłożone w [artykule z dokumentacji OpenCV](https://docs.opencv.org/3.4/d4/dee/tutorial_optical_flow.html), do którego lektury namawiam\*. Wiedzieć należy na pewno, że istnieją dwie główne odmiany metody przepływu optycznego:,
* przepływ gęsty (_dense_) - gdzie przesunięcie pomiędzy klatkami określane jest dla każdego piksela (na tej metodzie skupia się niniejsza część listy),
* przepływ rzadki (_sparse_) - gdzie ruch wykrywany jest tylko dla pewnego zbioru punktów zainteresowania w obrębie obrazu (przebadasz ją w drugiej części listy).

W ramach obu odmian występuje pewna liczba konkretnych metod obliczeniowych, w zależności od konkretnego podejścia do rozwiązywania równania ruchu. Na tych zajęciach wykorzystamy [algorytm Farnebacka](http://www.diva-portal.org/smash/get/diva2:273847/FULLTEXT01.pdf) - głównie dlatego, że jego gotowa implementacja znajduje się w pakiecie OpenCV.

\* - Czytając, zwróć uwagę na podział na sekcje _Lucas-Kanade_ oraz _Dense Optical Flow_. Analizując kod metody, poświęć chwilę na zrozumienie mapowania wyników do prezentowanego obrazu w przestrzeni HSV.

### Podejście

Większość algorytmów optical flow operuje na parze klatek, znajdując translację pomiędzy jedną a drugą. Jeśli więc interesuje nas przetwarzanie ciągłego strumienia wideo, praca przebiegać będzie na zasadzie dwuelementowej kolejki, tzn. zawsze patrzymy na klatkę obecną i poprzednią.

OpenCV oferuje banalnie prosty a zarazem potężny interfejs do obsługi strumieni wideo: [`cv2.VideoCapture`](https://docs.opencv.org/3.4/d8/dfe/classcv_1_1VideoCapture.html), za pomocą którego w ten sam sposób możemy obsługiwać pliki wideo w różnym kodowaniu, urządzenia wideo (np. kamerkę w laptopie) czy nawet wideo w protokole IP (choć występują pewne różnice z punktu widzenia użycia, jeśli korzystamy z zasobu hardware'owego działającego w czasie rzeczywistym). Idea jest prosta:
* tworzymy obiekt `cv2.VideoCapture` w odpowiedni sposób,
* pobieramy poszczególne klatki za pomocą metody [`VideoCapture::read`](https://docs.opencv.org/3.4/d8/dfe/classcv_1_1VideoCapture.html#a473055e77dd7faa4d26d686226b292c1).

Metoda `read` wykonuje całą pracę (odczytanie danych, dekodowanie strumienia wideo) i zwraca klatkę jako obraz w standardowym formacie OpenCV (a także flagę, czy w ogóle udało się pozyskać dane - krotka (flaga, klatka)). Zatem, aby pozyskać pierwszą klatkę z pliku wideo wystarczy:
```python
vid = cv2.VideoCapture("back.mp4")
r, frame = vid.read()
```

Drobnym ograniczeniem VideoCapture jest to, że nie ma możliwości cofnięcia się do poprzednio pobranej klatki (co jest naturalne w przypadku korzystania z fizycznego urządzenia do akwizycji, a może trochę mniej gdy czytamy z pliku). Jeśli potrzebny jest powrót do początku pliku wideo, niestety konieczne jest zamknięcie strumienia (`VideoCapture::release`) i ponowne otwarcie (`::open`).

Aby obliczyć (gęsty) przepływ optyczny pomiędzy dwiema klatkami, wykorzystamy algorytm Farnebacka, zaimplementowany w OpenCV w funkcji [`cv2.calcOpticalFlowFarneback`](https://docs.opencv.org/3.4/dc/d6b/group__video__track.html#ga5d10ebbd59fe09c5f650289ec0ece5af). Przyjmuje ona parę obrazów **w skali szarości**, opcjonalny argument `flow` (rozwiązanie można zainicjować poprzednio wyliczonym przepływem, jeśli nim dysponujemy), a następnie szereg parametrów sterujących metodą; m.in. można wykorzystać piramidyzację obrazów (rekomendowane `pyr_size` $=3$) czy określić rozmiar okna detekcji `winsize`. Sensowne pierwsze wartości dla argumentów podane są w dokumentacji.

Algorytm Farnebacka zwraca przepływ w formie obrazu o wymiarach przestrzennych równych obrazom wejściowym i dwóch kanałach, zawierających przesunięcie odpowiednio w osi $x$ i $y$. Można więc przetwarzać te dane dalej, np. obliczając kąt przesunięcia czy całkowitą odległość (i dalej, np. określić prędkość ruchu) - vide np. `cv2.cartToPolar`.

---

In [2]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from qwlist import Lazy
from typing import Iterable
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm

### Zadanie 1

#### Zadanie 1a
Otwórz wideo `kick.mp4` lub `back.mp4` i pobierz kilka klatek. Przewiń do interesującego Cię momentu - tak, aby uzyskać dwie klatki, na których widać ruch (pro-tip: znając framerate materiału (~25fps) i czas, w którym rozpoczyna się interesujący fragment, możesz w pętli "skonsumować" odpowiednią ilość klatek).  
Wykorzystaj algorytm Farnebacka do obliczenia przepływu pomiędzy klatkami. Wynik zaprezentuj w postaci obrazu całkowitego przesunięcia. W zależności od wybranego momentu w wideo, możesz spodziewać się uzyskania wyraźnych obszarów.

In [18]:
def get_video_details(video: cv2.VideoCapture):
    fps = video.get(cv2.CAP_PROP_FPS)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps
    return fps, frame_count, duration

def frame_pair_iter(path: str) -> Lazy[tuple[np.ndarray, np.ndarray]]:
    def inner_generator() -> Iterable[tuple[np.ndarray, np.ndarray]]:
        video1: cv2.VideoCapture = cv2.VideoCapture(path)
        video2: cv2.VideoCapture = cv2.VideoCapture(path)
        success, _ = video2.read()
        while success:
            _, frame1 = video1.read()
            success, frame2 = video2.read()
            if success:
                yield frame1, frame2
        video1.release()
        video2.release()
    return Lazy(inner_generator())

def make_gif(images: list[Image.Image], path: str, duration=200):
    images[0].save(
        path,
        save_all=True,
        append_images=images[1:],
        duration=duration,
        loop=0
    )

def ferneback(video_path: str, save_dir: str, skip: int = 0, duration: int = 200, winsize: int = 15):
    file_name = os.path.basename(video_path).split('.')[0]
    _, frame_count, _ = get_video_details(cv2.VideoCapture(video_path))
    cmap = LinearSegmentedColormap.from_list('custom', ['#FFE5E5', '#E0AED0', '#AC87C5', '#756AB6'])
    gen = frame_pair_iter(video_path).skip(skip)
    flow_frames_x = []
    flow_frames_y = []
    flow_frames = []
    canvases = []

    for frame1, frame2 in tqdm(gen, total=frame_count-2 - skip):
        frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        flow = cv2.calcOpticalFlowFarneback(
            prev=frame1_gray,
            next=frame2_gray,
            flow=None,
            pyr_scale=0.5,
            levels=3,
            winsize=winsize,
            iterations=3,
            poly_n=5,
            poly_sigma=1.2,
            flags=0
        )
        
        flow_frames.append(Image.fromarray(cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)))
        grad_x = flow[..., 0]
        grad_y = flow[..., 1]
        flow_frames_x.append(Image.fromarray(grad_x))
        flow_frames_y.append(Image.fromarray(grad_y))
        
        hsv = np.zeros_like(frame2)
        hsv[..., 1] = 255
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB).astype(np.uint8)

        h, w, _ = frame2.shape
        canvas = np.zeros((2 * h, w, 3))
        canvas[:h, ...] = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
        canvas[h:, ...] = rgb
        canvases.append(Image.fromarray(canvas.astype(np.uint8)))
    
    print('Saving results...')
    if not os.path.exists(f'{save_dir}/gradients_{file_name}'):
        os.makedirs(f'{save_dir}/gradients_{file_name}')
    make_gif(flow_frames, f'{save_dir}/gradients_{file_name}/original.gif', duration=duration)
    make_gif(flow_frames_x, f'{save_dir}/gradients_{file_name}/x.gif', duration=duration)
    make_gif(flow_frames_y, f'{save_dir}/gradients_{file_name}/y.gif', duration=duration)
    make_gif(canvases, f'{save_dir}/gradients_{file_name}/combination.gif', duration=duration)
    print('DONE')
     


In [16]:
ferneback('./data/shuttle.mp4', './gifs/gradients', skip=180)

  0%|          | 0/178 [00:00<?, ?it/s]

Saving results...
DONE


#### Zadanie 1b
Zbadaj wpływ parametru `winsize` na działanie metody.

In [ ]:
paths = [
    ('./data/back.mp4', 0),
    ('./data/kick.mp4', 0),
    ('./data/shot.mp4', 0)
]
winsizes = [3, 7, 15, 21, 31]
for path, skip in paths:
    for ws in winsizes:
        ferneback(path, f'gifs/winsize_param/size_{ws}', winsize=ws, skip=skip)

Komentarz 1:

...

---

### Zadanie 2

Powtórz powyższy eksperyment na materiale `shot.mp4` (przewiń materiał do momentu natychmiast po uderzeniu białej bili, ok. 20-25 klatek; framerate wynosi tu ok. 15fps).  
*W czym leży trudność? Co jest ograniczeniem metody?*

Komentarz 2:

...

### Zadanie dodatkowe

Powróć do takiego przypadku i konfiguracji algorytmu, dla którego uzyskane przez Ciebie wyniki były satysfakcjonujące. Przypomnij sobie zajęcia dotyczące np. segmentacji i wykorzystaj informację o przepływie optycznym do oddzielenia na obrazie obiektów od tła (metoda zupełnie dowolna).